In [1]:
import psycopg2
import pandas as pd   


In [2]:

try:
    conn = psycopg2.connect("dbname='himawari' user='postgres' host='localhost' password='newpassword'")
    print ("Connection Successful")

except:

    print ("I am unable to connect to the database")

#cur = conn.cursor()
#cur.execute("""SELECT * from transition_probs""")
#rows = cur.fetchall()
#activityGraphDf = pd.DataFrame(rows, columns=['from', 'to', 'probs'])

Connection Successful


In [3]:
cur = conn.cursor()
cur.execute("""SELECT "Policy Type Name", "combined_riders", "age", "Policyholder'S_Gender" from pi_ph_seg_seg2_seg3_insured_new as i 
where i."Termination  Reason" in ('G31','G32','G33','G34','G35','G36','G37')
        or i."Termination  Reason" in ('G41','G42','G43','G44')
        or i."Termination  Reason" in ('G52','G53')
        or i."Termination  Reason" in ('GA1','GA2')
        or i."Reduce Settlement Reason" = 'DA1'""")
rows = cur.fetchall()

In [4]:
policyHolders = pd.DataFrame(rows, columns=['policyTypeName', 'combinedRiders', 'age', 'gender'])

In [5]:
policyHolders.count()

policyTypeName    76768
combinedRiders    76768
age               76768
gender            76768
dtype: int64

In [6]:

genderList=[1,2]

ageRangesList=[[18,22],[23,30],[31,40],[41,50],[51,60],[61,64], [65]]

policyTypeList=policyHolders.policyTypeName.unique()


In [7]:
policy_type = pd.read_csv('/home/aizaz/ADDO/ADDO-AI/Himawari/Codes/PolicyTypeCode.csv')

In [8]:
policy_type

,Policy Type,Representative CD,Policy Type Name
0,100,10,Whole life
1,101,10,Whole life
2,102,10,Whole life
3,103,10,Whole life
4,104,10,Whole life
5,105,10,Whole life
6,106,10,Whole life
7,107,10,Whole life
8,108,10,Whole life
9,109,10,Whole life


In [9]:
#male=1, female=2

from collections import Counter
#words=["i", "love", "you", "i", "you", "a", "are", "you", "you", "fine", "green"]
#most_common_words= [word for word, word_count in Counter(words).most_common(3)]
#print most_common_words

mbaFile = open('/home/aizaz/ADDO/ADDO-AI/Himawari/Codes/MBA3_payout_colons','a') 

for gender in genderList:
    for ranges in ageRangesList:
        for policies in policyTypeList:
            
            if(len(ranges)==1):
                policyHoldersWithConditions=policyHolders[(policyHolders['gender']==str(gender)) &(policyHolders['policyTypeName']==str(policies)) &(policyHolders['age'] >= ranges[0])]
                policyHoldersWithConditions['combinedRidersList']=policyHoldersWithConditions['combinedRiders'].apply(lambda x: list(filter(None,x.split(',') )))

                policyHoldersWithConditions['numberOfRiders']=policyHoldersWithConditions['combinedRidersList'].apply(lambda x: len(x))
                pList=policyHoldersWithConditions['combinedRidersList'].tolist()
                dataset=[]
                datasetReplaced=[]
                for p in pList:
                    dataset=dataset + p
                    
                    
                for ds in dataset:
                    #print (ds[:3])
                    intermediateDf=policy_type[policy_type['Policy Type']==ds[:3]]
                    if (intermediateDf.shape[0]!=0):
                        value=intermediateDf['Policy Type Name'].values[0]
                        datasetReplaced= datasetReplaced + [value]
  
                    
                
                most_common_words= Counter(datasetReplaced).most_common(5)
                
                 
                meanRiders=str(policyHoldersWithConditions['numberOfRiders'].mean())
        
                printStr=str(gender)+':'+policies + ':' + '65 and above' + ':' + meanRiders + ':'+ str(most_common_words) + '\n'
                mbaFile.write(printStr) 
                print (printStr)

            
            else:

                policyHoldersWithConditions=policyHolders[(policyHolders['gender']==str(gender)) &(policyHolders['policyTypeName']==str(policies)) &(policyHolders['age'].between(ranges[0], ranges[1], inclusive=True))]

                policyHoldersWithConditions['combinedRidersList']=policyHoldersWithConditions['combinedRiders'].apply(lambda x: list(filter(None,x.split(',') )))

                policyHoldersWithConditions['numberOfRiders']=policyHoldersWithConditions['combinedRidersList'].apply(lambda x: len(x))
                pList=policyHoldersWithConditions['combinedRidersList'].tolist()
                dataset=[]
                datasetReplaced=[]
                for p in pList:
                    dataset=dataset + p
                for ds in dataset:
                    #print (ds[:3])
                    intermediateDf=policy_type[policy_type['Policy Type']==ds[:3]]
                    if (intermediateDf.shape[0]!=0):
                        value=intermediateDf['Policy Type Name'].values[0]
                        datasetReplaced= datasetReplaced + [value]
                    
                
                most_common_words= Counter(datasetReplaced).most_common(5)
                meanRiders=str(policyHoldersWithConditions['numberOfRiders'].mean())

                printStr=str(gender)+':'+policies + ':' + '('+ str (ranges[0]) + ',' +str (ranges[1])+ ')' + ':' + meanRiders + ':'+ str(most_common_words) + '\n'
            
                print (printStr)
                mbaFile.write(printStr)   
            
            

            
mbaFile.close()            
# break
    
            #policyHoldersWithConditions['length'] = policyHoldersWithConditions['itemsets'].apply(lambda x: len(x))
    
    

#policyHolders_Gender=policyHolders[policyHolders['gender']==str(genderList[0])]
#policyHolders_Age=policyHolders_Gender[policyHolders_Gender['age'].between(ageRangesList[0], ageRangesList[1], inclusive=True)]

#policyHolders_wholeLife=policyHolders_Age[policyHolders_Age['policyTypeName']==str(policyTypeList[0])]






/home/aizaz/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/aizaz/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1:Cancer:(18,22):0.0:[]

1:Term life:(18,22):1.15:[('Sickness Hospital', 10), ('Accidental Death', 7), ('Acc. Hospital', 5), ('M.I.P rider', 1)]

1:Medical:(18,22):1.6932515337423313:[('Medical Operation', 87), ('Acc. Hospital', 42), ('Advanced Medical', 41), ('Outpatient', 37), ('Adult Disease Hospital', 22)]

1:Endowment:(18,22):1.2194444444444446:[('Term rider', 137), ('Sickness Hospital', 111), ('Acc. Hospital', 106), ('Accidental Death', 85)]

1:Female Disease:(18,22):1.0:[('Female Operation', 4)]

1:Whole life:(18,22):1.75:[('Accidental Death', 71), ('Sickness Hospital', 69), ('Acc. Hospital', 64), ('Term rider', 5), ('Inc. Term rider', 1)]

1:M.I.P:(18,22):0.0:[]

1:Inc. Dec. Term Life:(18,22):nan:[]

1:VL Whole life:(18,22):0.3333333333333333:[('Accidental Death', 1)]

1:Dread Disease T.L.:(18,22):0.0:[]

1:Joint life:(18,22):6.0:[('Accidental Death', 2), ('Acc. Hospital', 2), ('Sickness Hospital', 2)]

1:Joint life(Flexible):(18,22):nan:[]

1:Annuity(Annuity Certain ):(18,22):

1:Joint life(Flexible):(51,60):2.0:[('Accidental Death', 4), ('Acc. Hospital', 2), ('Sickness Hospital', 2)]

1:Annuity(Annuity Certain ):(51,60):0.0:[]

1:Dread Disease W.L.:(51,60):0.22448979591836735:[('Family Medical', 33)]

1:Juvenile:(51,60):nan:[]

1:VL Endowment:(51,60):0.36363636363636365:[('Accidental Death', 2), ('Acc. Hospital', 1), ('Sickness Hospital', 1)]

1:Injury:(51,60):1.0:[('Acc. Hospital', 1)]

1:Nursing care:(51,60):0.0:[]

1:Annuity(Whole Life Annuity With Guaranteed Installment ):(51,60):nan:[]

1:Cancer:(61,64):0.04252199413489736:[('Advanced Medical', 17), ('Cancer Dia', 6), ('Cancer Death', 2), ('Female Operation', 2), ('Family Cancer', 2)]

1:Term life:(61,64):0.7804428044280443:[('Sickness Hospital', 270), ('Acc. Hospital', 76), ('Accidental Death', 56), ('M.I.P rider', 10), ('Inc. Term rider', 5)]

1:Medical:(61,64):1.3055403181568843:[('Medical Operation', 781), ('Advanced Medical', 424), ('Acc. Hospital', 379), ('Outpatient', 250), ('Cancer Hospital', 21

/home/aizaz/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/aizaz/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1:Term life:65 and above:0.5657311669128509:[('Sickness Hospital', 294), ('Acc. Hospital', 66), ('Accidental Death', 18), ('Adult Disease Hospital', 2), ('M.I.P rider', 2)]

1:Medical:65 and above:1.0853979968371112:[('Advanced Medical', 668), ('Medical Operation', 367), ('Adult Disease Hospital', 242), ('Acc. Hospital', 231), ('Cancer Hospital', 204)]

1:Endowment:65 and above:0.7551020408163265:[('Sickness Hospital', 17), ('Acc. Hospital', 15), ('Term rider', 5)]

1:Female Disease:65 and above:1.0:[('Female Operation', 1)]

1:Whole life:65 and above:0.17139001349527666:[('Sickness Hospital', 119), ('Acc. Hospital', 114), ('Term rider', 8), ('Accidental Death', 6), ('Adult Disease Hospital', 3)]

1:M.I.P:65 and above:0.08333333333333333:[('Accidental Death', 2)]

1:Inc. Dec. Term Life:65 and above:0.0:[]

1:VL Whole life:65 and above:0.21153846153846154:[('Acc. Hospital', 5), ('Sickness Hospital', 5), ('Accidental Death', 1)]

1:Dread Disease T.L.:65 and above:0.0:[]

1:Joint life:65 

2:Whole life:(51,60):0.896361631753032:[('Sickness Hospital', 324), ('Acc. Hospital', 307), ('Accidental Death', 164), ('Term rider', 8), ('Adult Disease Hospital', 5)]

2:M.I.P:(51,60):0.0:[]

2:Inc. Dec. Term Life:(51,60):0.0:[]

2:VL Whole life:(51,60):0.7142857142857143:[('Accidental Death', 6), ('Acc. Hospital', 6), ('Sickness Hospital', 6), ('Adult Disease Hospital', 1), ('Term rider', 1)]

2:Dread Disease T.L.:(51,60):0.0:[]

2:Joint life:(51,60):3.2:[('Acc. Hospital', 6), ('Sickness Hospital', 6), ('Accidental Death', 4)]

2:Joint life(Flexible):(51,60):1.0:[('Acc. Hospital', 1), ('Sickness Hospital', 1)]

2:Annuity(Annuity Certain ):(51,60):0.03508771929824561:[('Acc. Hospital', 1), ('Sickness Hospital', 1)]

2:Dread Disease W.L.:(51,60):0.15492957746478872:[('Family Medical', 11)]

2:Juvenile:(51,60):nan:[]

2:VL Endowment:(51,60):0.0:[]

2:Injury:(51,60):1.0:[('Acc. Hospital', 1)]

2:Nursing care:(51,60):0.0:[]

2:Annuity(Whole Life Annuity With Guaranteed Installment ):(51,